##### Copyright 2025 Google LLC.

In [ ]:
# @title Licenciado bajo la Licencia Apache, Versión 2.0 (la "Licencia");
# no puedes usar este archivo excepto en cumplimiento con la Licencia.
# Puedes obtener una copia de la Licencia en
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# A menos que lo requiera la ley aplicable o se acuerde por escrito, el software
# distribuido bajo la Licencia se distribuye "TAL CUAL",
# SIN GARANTÍAS NI CONDICIONES DE NINGÚN TIPO, ya sean expresas o implícitas.
# Consulta la Licencia para conocer el lenguaje específico que rige los permisos y
# limitaciones bajo la Licencia.

# Día 2 - Preguntas y Respuestas con Documentos usando RAG y Chroma

¡Bienvenido de nuevo al curso de 5 días de IA Generativa en Kaggle!

**NOTA**: El cuaderno del Día 1 contiene mucha información para configurar los cuadernos de Kaggle. Si tienes algún problema, por favor [consulta los pasos de solución de problemas allí](https://www.kaggle.com/code/markishere/day-1-prompting#Get-started-with-Kaggle-notebooks).

Dos grandes limitaciones de los LLMs son 1) que solo "conocen" la información con la que fueron entrenados, y 2) que tienen ventanas de contexto de entrada limitadas. Una forma de abordar ambas limitaciones es usar una técnica llamada Generación Aumentada con Recuperación, o RAG. Un sistema RAG tiene tres etapas:

1. Indexación
2. Recuperación
3. Generación

La indexación ocurre con anticipación y te permite buscar información relevante rápidamente en el momento de la consulta. Cuando llega una consulta, recuperas documentos relevantes, los combinas con tus instrucciones y la consulta del usuario, y haces que el LLM genere una respuesta personalizada en lenguaje natural utilizando la información proporcionada. Esto te permite proporcionar información que el modelo no ha visto antes, como conocimiento específico de productos o actualizaciones meteorológicas en vivo.

En este cuaderno usarás la API de Gemini para crear una base de datos vectorial, recuperar respuestas a preguntas desde la base de datos y generar una respuesta final. Usarás [Chroma](https://docs.trychroma.com/), una base de datos vectorial de código abierto. Con Chroma, puedes almacenar embeddings junto con metadatos, incrustar documentos y consultas, y buscar en tus documentos.

## Para ayuda

**Los problemas comunes están cubiertos en la [guía de preguntas frecuentes y solución de problemas](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs).**


## Configuración

Primero, instala ChromaDB y el SDK de Python de la API de Gemini.

In [ ]:
!pip uninstall -qqy jupyterlab kfp  # Elimina paquetes conflictivos no utilizados
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [15]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

### Configura tu clave API

Para ejecutar la siguiente celda, tu clave API debe estar almacenada en un [secreto de Kaggle](https://www.kaggle.com/discussions/product-feedback/114053) llamado `GOOGLE_API_KEY`.

Si aún no tienes una clave API, puedes obtener una desde [AI Studio](https://aistudio.google.com/app/apikey). Puedes encontrar [instrucciones detalladas en la documentación](https://ai.google.dev/gemini-api/docs/api-key).

Para hacer que la clave esté disponible a través de secretos de Kaggle, elige `Secrets` desde el menú `Add-ons` y sigue las instrucciones para agregar tu clave o habilitarla para este cuaderno.

In [16]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

Si recibiste un mensaje de error como `No user secrets exist for kernel id ...`, entonces necesitas agregar tu clave API a través de `Add-ons`, `Secrets` **y** habilitarla.

![Captura de pantalla de la casilla para habilitar el secreto GOOGLE_API_KEY](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explora los modelos disponibles

Usarás el método de API [`embedContent`](https://ai.google.dev/api/embeddings#method:-models.embedcontent) para calcular embeddings en esta guía. Encuentra un modelo que lo soporte a través del endpoint [`models.list`](https://ai.google.dev/api/models#method:-models.list). También puedes encontrar más información sobre los modelos de embeddings en [la página de modelos](https://ai.google.dev/gemini-api/docs/models/gemini#text-embedding).

`text-embedding-004` es el modelo de embeddings más reciente disponible en general, por lo que lo usarás para este ejercicio, pero también prueba el modelo experimental `gemini-embedding-exp-03-07`.

In [17]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### Datos

Aquí tienes un pequeño conjunto de documentos que usarás para crear una base de datos de embeddings.

In [ ]:
DOCUMENT1 = "Operación del Sistema de Control de Clima  Tu Googlecar tiene un sistema de control de clima que te permite ajustar la temperatura y el flujo de aire en el coche. Para operar el sistema de control de clima, usa los botones y perillas ubicados en la consola central.  Temperatura: La perilla de temperatura controla la temperatura dentro del coche. Gira la perilla en el sentido de las agujas del reloj para aumentar la temperatura o en sentido contrario para disminuirla. Flujo de aire: La perilla de flujo de aire controla la cantidad de flujo de aire dentro del coche. Gira la perilla en el sentido de las agujas del reloj para aumentar el flujo de aire o en sentido contrario para disminuirlo. Velocidad del ventilador: La perilla de velocidad del ventilador controla la velocidad del ventilador. Gira la perilla en el sentido de las agujas del reloj para aumentar la velocidad del ventilador o en sentido contrario para disminuirla. Modo: El botón de modo te permite seleccionar el modo deseado. Los modos disponibles son: Auto: El coche ajustará automáticamente la temperatura y el flujo de aire para mantener un nivel cómodo. Cool: El coche soplará aire fresco en el coche. Heat: El coche soplará aire caliente en el coche. Defrost: El coche soplará aire caliente en el parabrisas para descongelarlo."
DOCUMENT2 = 'Tu Googlecar tiene una gran pantalla táctil que proporciona acceso a una variedad de funciones, incluyendo navegación, entretenimiento y control de clima. Para usar la pantalla táctil, simplemente toca el ícono deseado.  Por ejemplo, puedes tocar el ícono de "Navegación" para obtener direcciones a tu destino o tocar el ícono de "Música" para reproducir tus canciones favoritas.'
DOCUMENT3 = "Cambio de Marchas Tu Googlecar tiene una transmisión automática. Para cambiar de marcha, simplemente mueve la palanca de cambios a la posición deseada.  Park: Esta posición se usa cuando estás estacionado. Las ruedas están bloqueadas y el coche no puede moverse. Reverse: Esta posición se usa para retroceder. Neutral: Esta posición se usa cuando estás detenido en un semáforo o en el tráfico. El coche no está en marcha y no se moverá a menos que presiones el pedal del acelerador. Drive: Esta posición se usa para avanzar. Low: Esta posición se usa para conducir en nieve u otras condiciones resbaladizas."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## Creando la base de datos de embeddings con ChromaDB

Crea una [función personalizada](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) para generar embeddings con la API de Gemini. En esta tarea, estás implementando un sistema de recuperación, por lo que el `task_type` para generar los embeddings de *documentos* es `retrieval_document`. Más adelante, usarás `retrieval_query` para los embeddings de *consultas*. Consulta la [referencia de la API](https://ai.google.dev/api/embeddings#v1beta.TaskType) para la lista completa de tareas soportadas.

Palabras clave: Los documentos son los elementos que están en la base de datos. Se insertan primero y luego se recuperan. Las consultas son los términos de búsqueda textuales y pueden ser palabras clave simples o descripciones textuales de los documentos deseados.

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define un ayudante para reintentar cuando se alcance la cuota por minuto.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) y e.code en {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Especifica si generar embeddings para documentos o consultas
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Ahora crea un [cliente de base de datos Chroma](https://docs.trychroma.com/getting-started) que use la `GeminiEmbeddingFunction` y llena la base de datos con los documentos que definiste anteriormente.

In [20]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

Confirma que los datos fueron insertados mirando la base de datos.

In [ ]:
db.count()
# También puedes echar un vistazo a los datos.
# db.peek(1)

3

## Recuperación: Encuentra documentos relevantes

Para buscar en la base de datos de Chroma, llama al método `query`. Ten en cuenta que también cambias al modo `retrieval_query` de generación de embeddings.


In [ ]:
# Cambia al modo de consulta al generar embeddings.
embed_fn.document_mode = False

# Busca en la base de datos de Chroma usando la consulta especificada.
query = "¿Cómo usas la pantalla táctil para reproducir música?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

## Generación aumentada: Responde la pregunta

Ahora que has encontrado un pasaje relevante del conjunto de documentos (el paso de *recuperación*), ahora puedes ensamblar un prompt de generación para que la API de Gemini *genere* una respuesta final. Ten en cuenta que en este ejemplo solo se recuperó un pasaje. En la práctica, especialmente cuando el tamaño de tus datos subyacentes es grande, querrás recuperar más de un resultado y dejar que el modelo de Gemini determine qué pasajes son relevantes para responder la pregunta. Por esta razón, está bien si algunos pasajes recuperados no están directamente relacionados con la pregunta: este paso de generación debería ignorarlos.

In [ ]:
query_oneline = query.replace("\n", " ")

# Este prompt es donde puedes especificar cualquier orientación sobre el tono, o qué temas debe seguir o evitar el modelo.
prompt = f"""Eres un bot útil e informativo que responde preguntas usando texto del pasaje de referencia incluido a continuación. 
Asegúrate de responder en una oración completa, siendo comprensivo, incluyendo toda la información de fondo relevante. 
Sin embargo, estás hablando con una audiencia no técnica, así que asegúrate de desglosar conceptos complicados y 
mantener un tono amigable y conversacional. Si el pasaje es irrelevante para la respuesta, puedes ignorarlo.

PREGUNTA: {query_oneline}
"""

# Agrega los documentos recuperados al prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASAJES: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



Ahora usa el método `generate_content` para generar una respuesta a la pregunta.

In [24]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

To play your favorite songs on the touchscreen display in your Googlecar, simply touch the "Music" icon, which will then allow you to access your favorite songs.


## Próximos pasos

¡Felicidades por construir una aplicación de Generación Aumentada con Recuperación!

Para aprender más sobre el uso de embeddings en la API de Gemini, consulta la [Introducción a los embeddings](https://ai.google.dev/gemini-api/docs/embeddings) o para aprender más fundamentos, estudia el [capítulo de embeddings](https://developers.google.com/machine-learning/crash-course/embeddings) del Curso Intensivo de Aprendizaje Automático.

Para un sistema RAG alojado, consulta el [servicio de Recuperación Semántica](https://ai.google.dev/gemini-api/docs/semantic_retrieval) en la API de Gemini. Puedes implementar preguntas y respuestas en tus propios documentos en una sola solicitud, o alojar una base de datos para respuestas aún más rápidas.

*- [Mark McD](https://linktr.ee/markmcd)*